把结果填入到eta_task_answer.csv里面

In [5]:
import pandas as pd
import requests
from datetime import timedelta

eta_task = pd.read_csv('data/result_eta_task.csv')
eta_answer = pd.read_csv('data/eta_answer.csv')


In [6]:
eta_answer_dict = dict(zip(eta_answer['tid'], eta_answer['timd']))

# 使用 apply 函数将分钟转换为 timedelta 类型
eta_task['timd'] = eta_task['traj_id'].apply(lambda x: timedelta(minutes=float(eta_answer_dict.get(x, 0))))

# 将时间字符串转换为 datetime 对象
eta_task['eta'] = pd.to_datetime(eta_task['time'])

# 在计算新的时间戳前检查是否存在时间差
eta_task['eta'] = eta_task['eta'] + eta_task['timd']

# 格式化输出时间戳
eta_task['formatted_eta'] = eta_task['eta'].dt.strftime('%Y-%m-%d %H:%M:%S')

# 打印结果
print(eta_task[['traj_id', 'time', 'formatted_eta']])

      traj_id                  time        formatted_eta
0           9  2013-10-08T22:30:00Z  2013-10-08 22:36:20
1          11  2013-10-08T23:00:00Z  2013-10-08 23:06:39
2          13  2013-10-08T17:15:00Z  2013-10-08 17:15:00
3          16  2013-10-08T19:00:00Z  2013-10-08 19:11:43
4          24  2013-10-08T08:00:00Z  2013-10-08 08:19:18
...       ...                   ...                  ...
4395    21983  2013-10-30T09:45:00Z  2013-10-30 09:53:54
4396    21984  2013-10-30T12:30:00Z  2013-10-30 12:37:34
4397    21991  2013-10-30T19:15:00Z  2013-10-30 19:26:04
4398    21997  2013-10-30T22:00:00Z  2013-10-30 22:14:00
4399    21999  2013-10-30T00:00:00Z  2013-10-30 00:09:19

[4400 rows x 3 columns]


存起来

In [7]:
eta_task.to_csv('answer/eta_task_result.csv', index=False)

In [8]:
import ast

def getETA(task_id):
    d = eta_task[eta_task['traj_id'] == task_id]
    return ast.literal_eval(d['from'].values[0]), ast.literal_eval(d['to'].values[0])


def calculate_driving_time(traj_id):
    api_key = '8122351ca4498dd9af86b6b1f21c6a47'
    origin, destination = getETA(traj_id)
    origin = str(str(origin[0]) + ',' + str(origin[1]))
    destination = str(str(destination[0])+','+str(destination[1]))
    print(origin, destination)
    url = "https://restapi.amap.com/v5/direction/driving"
    params = {
        "key": api_key,
        "origin": origin,
        "destination": destination,
        "output": "json",
        "strategy": 0,  # 0: 速度优先
        "show_fields" : 'cost'
    }

    response = requests.get(url, params=params)
    data = response.json()
    # print(data)

    if data.get("status") == "1" and int(data.get("count", 0)) > 0:
        total_duration = sum(int(step.get('cost', 0).get("duration", 0)) for path in data["route"]["paths"] for step in path["steps"])
        return (total_duration / 60)
    else:
        print("Error in API response")
        return None

# 删除之前的 sum 变量

# 重新定义新的变量
total_sum = calculate_driving_time(traj_id=13)
print(total_sum)


116.335503,39.763187 116.34182,39.771339


4.916666666666667


下面是直接操作之前的`eta_task.csv`

In [9]:
df = pd.read_csv('data/eta_task.csv')

gaode = 0

for i in range(len(df)):
    if i % 2 == 0:
        continue
    if df.loc[i, 'traj_id'] not in eta_answer_dict or eta_answer_dict[df.loc[i, 'traj_id']] == 2.0:
        print(df.loc[i, 'traj_id'])
        timd_minutes_val = calculate_driving_time(traj_id=int(df.loc[i, 'traj_id']))
        timd_timedelta = pd.to_timedelta(timd_minutes_val, unit='m')
        gaode = gaode + 1
        df.loc[i, 'time'] = (pd.to_datetime(df.loc[i-1, 'time']) + timd_timedelta).strftime('%Y-%m-%dT%H:%M:%SZ')
        ## 这里交给高德
    elif df.loc[i, 'traj_id'] in eta_answer_dict and pd.isna(df.loc[i, 'time']):
        timd_minutes = eta_answer_dict[df.loc[i, 'traj_id']]
        timd_timedelta = pd.to_timedelta(timd_minutes, unit='m')
        df.loc[i, 'time'] = (pd.to_datetime(df.loc[i-1, 'time']) + timd_timedelta).strftime('%Y-%m-%dT%H:%M:%SZ')



print(df[['traj_id', 'time']])

print(gaode)

13
116.335503,39.763187 116.34182,39.771339
97
116.316399,39.978687 116.313873,40.024509
105
116.461601,39.940952 116.417839,40.024811
236
116.513145,40.010815 116.543999,40.025146
336
116.351448,40.024452 116.352158,40.007809
436
116.547768,39.965778 116.530746,39.91703
441
116.305901,39.980801 116.314201,40.020992
488
116.226585,39.907337 116.198799,39.913048
539
116.379509,39.938744 116.348625,40.025135
560
116.322876,39.896828 116.517105,39.916298
680
116.534676,39.809269 116.512383,39.795067
806
116.288101,40.006935 116.286621,40.014984
923
116.477928,39.903603 116.54718,39.923576
974
116.237366,39.954517 116.228767,39.954636
1353
116.384621,39.987488 116.412781,40.000614
1378
116.375259,39.864914 116.547691,39.923557
1560
116.252808,39.906548 116.244949,39.89978
1591
116.468803,39.986099 116.457832,40.004452
1630
116.332695,39.890049 116.314163,40.02248
1717
116.33522,39.89299 116.352821,39.88942
2079
116.34507,39.91143 116.32618,39.893749
2158
116.250435,39.947292 116.329933,39.

In [10]:
df.to_csv('answer/eta_task.csv', index=False)

In [11]:
df

,id,traj_id,time,entity_id,coordinates,current_dis,speeds,holidays
0,126,9,2013-10-08T22:30:00Z,254,"[116.490013,39.892902]",0.000000,37.2925,0
1,133,9,2013-10-08T22:36:20Z,254,"[116.511292,39.904655]",3.079017,24.6500,0
2,147,11,2013-10-08T23:00:00Z,254,"[116.450546,39.943771]",0.000000,36.0575,0
3,164,11,2013-10-08T23:06:39Z,254,"[116.440742,40.018944]",12.543428,49.9000,0
4,171,13,2013-10-08T17:15:00Z,255,"[116.335503,39.763187]",0.000000,26.6250,0
...,...,...,...,...,...,...,...,...
8795,518214,21991,2013-10-30T19:26:04Z,4954,"[116.371025,39.84676]",7.175550,17.9425,0
8796,518315,21997,2013-10-30T22:00:00Z,4954,"[116.324913,39.938717]",0.000000,39.0125,0
8797,518348,21997,2013-10-30T22:14:00Z,4954,"[116.2817,39.945621]",11.553799,25.0725,0
8798,518408,21999,2013-10-30T00:00:00Z,4955,"[116.301109,39.985023]",0.000000,55.1025,0
